<a href="https://colab.research.google.com/github/TuyetMaiLe/COVID19-Fake-News-Detection-in-English/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SỬ DỤNG GPU

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch
if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


#CÀI ĐẶT THƯ VIỆN

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 7.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 2.9MB 9.1MB/s 
     |████████████████████████████████| 1.8MB 20.3MB/s 
     |████████████████████████████████| 317kB 54.9MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
     |████████████████████████████████| 7.4MB 51.1MB/s 
     |████████████████████████████████| 1.4MB 18.9MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 163kB 48.0MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 133kB 49.2MB/s 
     |████████████████████████████████| 112kB 53.9MB/s 
     |████████████████████████████████| 4.5MB 17.7MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 122kB 54.3MB/s 

In [ ]:
!pip install --upgrade simpletransformers

In [ ]:
#!wget https://public.vinai.io/BERTweet_base_transformers.tar.gz

In [ ]:
#!tar -xzvf BERTweet_base_transformers.tar.gz

In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49715 sha256=9286ea5c832bccc321189b072ea8a37b12b33604ef3ec13d8c774405367de48a
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init (project = 'Covid19_PROJECT', name = 'normal')

wandb: Currently logged in as: tuyetmaile (use `wandb login --relogin` to force relogin)


#DATA và TIỀN XỬ LÝ


##Ghi nhật ký

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("wandb")
logger.warning("Is this working?") 
logger.setLevel(logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [ ]:
import pandas as pd
df = pd.read_excel('/content/Constraint_English_Train.xlsx')
df.loc[:,'tweet'] = df['tweet'].apply(lambda x: normalizeTweet(x))
df.loc[:,"label"] = df['label'].apply(lambda x: 0 if x == 'fake' else 1)
df = df.drop(columns='id')
df.columns = ["text", "label"]

#TOKENIZER

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-covid19-base-cased", normalization=True, use_fast=False, do_lower_case=True)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
tweets = train_df.tweet.values
tokenized_texts = [tokenizer.tokenize(sent) for sent in tweets]

In [ ]:
print (tokenized_texts)

[['The', 'CDC', 'currently', 'reports', '99031', 'deaths', '.', 'In', 'general', 'the', 'discrepancies', 'in', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'explicable', '.', 'The', 'death', 'toll', 'stands', 'at', 'roughly', '100000', 'people', 'today', '.'], ['States', 'reported', '1121', 'deaths', 'a', 'small', 'rise', 'from', 'last', 'Tuesday', '.', 'Southern', 'states', 'reported', '640', 'of', 'those', 'deaths', '.', 'HTTPURL'], ['Politically', 'Correct', 'Woman', '(', 'Almost', ')', 'Uses', 'Pandemic', 'as', 'Excuse', 'Not', 'to', 'Reuse', 'Plastic', 'Bag', 'HTTPURL', '#coronavirus', '#nashville'], ['#IndiaFightsCorona', ':', 'We', 'have', '1524', '#COVID', 'testing', 'laboratories', 'in', 'India', 'and', 'as', 'on', '25th', 'August', '2020', '3682752', '0', 'tests', 'have', 'been', 'done', ':', '@USER', 'DG', '@USER', '#StaySafe', '#IndiaWillWin', 'HTTPURL'], ['Populous', 'states', 'can', 'generate', 'large', 'case', 'counts', 'but', 'if', 'you',

In [ ]:
input_ids = torch.tensor([tokenizer.encode(tweet)])

with torch.no_grad():
    features = bertweet(input_ids) 

In [ ]:
from transformers import BertweetTokenizer
tokenizer = BertweetTokenizer.from_pretrained("vinai/bertweet-covid19-base-cased", normalization=True, use_fast=False, do_lower_case=True)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
input_ids = []
attention_masks = []

# For every sentence...
for sent in tweet:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(sent, max_length = 100, pad_to_max_length = True, 
                        return_attention_mask = True,  
                        return_tensors = 'pt' )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label)


In [ ]:
print('Original: ', tweet[0])
print('Token IDs:', input_ids[0])

# PHÂN CHIA DỮ LIỆU

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, random_state=0, test_size=0.2)

# FINE TURNING



## LOAD PRETRAIN-MODEL BERTweet

In [ ]:
from simpletransformers.classification import ClassificationModel
args = {'reprocess_input_data': True, 'overwrite_output_dir': True}
model = ClassificationModel("bertweet", "vinai/bertweet-covid19-base-cased", num_labels=2, args=args)


Some weights of the model checkpoint at vinai/bertweet-covid19-base-cased were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-covid19-base-cased and are newly initialized: ['classifier.dense.weight', 'cla

In [ ]:
# Kiểm tra các tham số mặc định quan trọng
print ('type arg = ',type(model.args))
print ('train_batch_size = ' ,model.args.train_batch_size)
print ('num_train_epochs = ',model.args.num_train_epochs)
print ('max_seq_length', model.args.max_seq_length)
print ('learning_rate = ', model.args.learning_rate)
print ('model_name = ', model.args.model_name)


type arg =  <class 'simpletransformers.config.model_args.ClassificationArgs'>
train_batch_size =  8
num_train_epochs =  1
max_seq_length 128
learning_rate =  4e-05
model_name =  vinai/bertweet-covid19-base-cased


## Kiểm tra đối số đào tạo

In [ ]:
type(model.args)

simpletransformers.config.model_args.ClassificationArgs

In [ ]:
arg_values = []

pd.set_option('display.max_rows', 80)

for arg in dir(model.args):
    
    if (not arg[0:2] == '__') and (not callable(getattr(model.args, arg))):
        arg_values.append((arg, str(getattr(model.args, arg))))
df_args = pd.DataFrame(arg_values)        
df_args.head(88)

,0,1
0,adafactor_beta1,None
1,adafactor_clip_threshold,1.0
2,adafactor_decay_rate,-0.8
3,adafactor_eps,"(1e-30, 0.001)"
4,adafactor_relative_step,True
...,...,...
83,wandb_kwargs,{}
84,wandb_project,None
85,warmup_ratio,0.06
86,warmup_steps,0


## Thay đổi tham số quan trọng


Training


In [ ]:
model.args.max_seq_length = 128
model.args.num_train_epochs = 4
model.args.train_batch_size = 32
model.args.learning_rate = 2e-5

In [ ]:
import numpy as np

steps_per_epoch = int(np.ceil(len(train_df) / float(model.args.train_batch_size)))

print('Each epoch will have {:,} steps.'.format(steps_per_epoch))

Each epoch will have 161 steps.


## early stop

In [ ]:
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 2


In [ ]:
# Chạy đánh giá trong suốt quá trình đào tạo
model.args.evaluate_during_training = True
# In đánh giá
model.args.evaluate_during_training_verbose = True
# Thực hiện đánh giá ở bước cụ thể
model.args.evaluate_during_training_steps = 32

In [ ]:
model_args.evaluate_during_training_steps = 20
model_args.early_stopping_patience = 10
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = False

In [ ]:
print('Training on {:,} samples...'.format(len(train_df)))

out = model.train_model(train_df, eval_df=validation_df)

Đánh giá trên tập test

In [ ]:
model = ClassificationModel("bertweet", "outputs/best_model")

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bertweet_128_2_1284


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9332039099154181, 'tp': 643, 'tn': 598, 'fp': 29, 'fn': 14, 'eval_loss': 0.12436168858642627}


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.948526713044759, 'tp': 1099, 'tn': 986, 'fp': 34, 'fn': 21, 'eval_loss': 0.08800298479340497}


chạy thu 3 lan

In [ ]:
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split

model_args.wandb_project = 'Best Checkpoint 5'
model_args.wandb_kwargs = {}
args = {'reprocess_input_data': True, 'overwrite_output_dir': True}
for trial_num in range(0, 10):
    run_name ='Trial ' + str(trial_num)

   
    #run_name = 'max_len {:} - batch_size {:} - lr {:} - epochs {:}'.format(model_args.max_seq_length, model_args.train_batch_size, model_args.learning_rate, model.args.num_train_epochs)

    print('============================')
    print('       ', run_name)
    print('============================')

    # This `wandb_kwargs` dictionary will get passed to `wandb.init()`, allowing
    # us to set additional parameters such as our custom run name.
    model_args.wandb_kwargs = {'name': run_name,}

    df_train_80, validation_df = train_test_split(train_df, test_size=0.2)
    model = ClassificationModel("bertweet", "vinai/bertweet-covid19-base-cased", num_labels=2, args=args)

    print('Training on {:,} samples...'.format(len(df_train_80)))

    # =====================
    #       Train
    # =====================

    # Train the model, testing against the validation set periodically.
    model.train_model(df_train_80, eval_df=validation_df)
    #model.args.wandb_kwargs['id'] = wandb.run.id
    #model.args.wandb_kwargs['resume'] = wandb.run.id

    result, model_outputs, wrong_predictions = model.eval_model(validation_df)
    wandb.init (project = 'Covid19_CTBERT', name = 'train_batch_size = 32_learning_rate = 2e-5')
    wandb.log({'end_val_mcc': result['mcc']})

    # Test and record Test set.
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    wandb.log({'end_test_mcc': result['mcc']})
    model = ClassificationModel("bertweet", "outputs/best_model")
    #model.args.wandb_kwargs['id'] = wandb.run.id
    #model.args.wandb_kwargs['resume'] = wandb.run.id

    # Test and record Validation set.
    result, model_outputs, wrong_predictions = model.eval_model(validation_df) # , wandb_log=False)
    wandb.log({'best_val_mcc': result['mcc']})

    # Test and record Test set.
    result, model_outputs, wrong_predictions = model.eval_model(test_df) # , wandb_log=False)
    wandb.log({'best_test_mcc': result['mcc']})
    wandb.finish()

        Trial 0


Some weights of the model checkpoint at vinai/bertweet-covid19-base-cased were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-covid19-base-cased and are newly initialized: ['classifier.dense.weight', 'cla

Training on 4,108 samples...


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bertweet_128_2_1028


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9863634894405717, 'tp': 546, 'tn': 475, 'fp': 1, 'fn': 6, 'eval_loss': 0.02395768912306936}
INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bertweet_128_2_2140


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9419460447782159, 'tp': 1087, 'tn': 991, 'fp': 29, 'fn': 33, 'eval_loss': 0.09629721354707074}


end_val_mcc,0.94327
_step,3
_runtime,25
_timestamp,1607593607
end_test_mcc,0.94194
best_val_mcc,0.98636
best_test_mcc,0.94195


end_val_mcc,▁
_step,▁▃▆█
_runtime,▁▄▆█
_timestamp,▁▄▆█
end_test_mcc,▁
best_val_mcc,▁
best_test_mcc,▁


In [ ]:
from dataclasses import asdict

if False:
    # Tell wandb to reconnect to the same "run" used for training above.
    wandb.init(project=model.args.wandb_project, # Must specify the project name.
            config={**asdict(model.args)}, # Store all of the model configuration parameters, just for the record.
            **wandb_kwargs) # Specifies the name and run ID.

# ĐÁNH GIÁ 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [ ]:
def Ham_danh_gia:
  print('accuracy ',accuracy_score(y_test,Y_pred))
  print('precision ', precision_score(y_test,Y_pred,average= 'weighted'))
  print('recall ', recall_score(y_test,Y_pred,average= 'weighted'))
  print("f1", f1_score(y_test,Y_pred, average= 'weighted'))
  print(classification_report(y_test, Y_pred, target_names = ["FAKE","REAL"]))
  confusion_matrix(y_test,Y_pred, labels=[0,1])

In [ ]:
df_test = pd.read_excel('/content/Constraint_English_Val.xlsx')
df_test.loc[:,"label"] = df_test['label'].apply(lambda x: 0 if x == 'fake' else 1)
df_test = df_test.drop(columns='id')
df_test.columns = ["text", "label"]